In [1]:
#иной подход к TEA
#1) Ставим адаптер в параллель с последним трансформерным слоем. Адаптер - это трансформерные слои + резнеты между ними
#2) Возможно, ставим адаптер ещё и в параллель с оригинальной LM-head
#Причём линейную LM-head можно как обучать, так и не обучать
#Почему-то лоссы выходят не очень. Чем больше обучаем, тем более "не очень".
#Поэтому полагаю, верный пайплайн такой: обучили и трансформерные адаптеры, и резнет на выходе (train_transformer = True, train_lm_head = True). 
#А потом, когда трансформерные слои научились выделять интересные нам фичи и делать эмбеддинг более репрезентативным,
#Ставим train_transformer = False, train_lm_head = True - и дотягиваем последний адаптер
#Специально стараюсь сделать, чтобы выходной адаптер был побольше, а трансформерные - поменьше. 
#Потому что затраты на выполнение трансформерных = O(N^2), где N - длина входной последовательности. А у выходного резнета сложность O(1)

In [2]:
install = False
#Через requirements поставить это затруднительно, последовательность установки важна, так как часть либ компилируются
#Эта последовательность установки, как бы странно она ни выглядела, работает. Не в любых окружениях, но во многих.
if install:

    !pip install deepspeed==0.14.4

    !pip install --upgrade pip
    !pip install --upgrade transformers==4.46.2 tyro==0.9.8 triton==2.3.1 trl==0.12.0

    !pip install -v "xformers==0.0.29.post1"
    #!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
    !pip install --no-deps -v "peft==0.13.2"
    !pip install --no-deps packaging ninja einops flash-attn bitsandbytes==0.44.1
    !pip install accelerate==0.34.2
    !pip uninstall peft -y
    !pip install -v "peft==0.13.2"

In [3]:
import sys
import time
import json
import os
from collections import OrderedDict
from pathlib import Path
from collections import deque
import os, pickle, random
import psutil
import warnings
import codecs
import numpy as np
import pandas as pd
import requests
from typing import Dict, List, Optional
import torch.nn as nn
import torch.nn.functional as F

import torch
import torch.nn.functional as F

from torch.utils.data import Dataset, DataLoader

import transformers
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModel
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
from peft import PeftModel, PeftConfig, PeftModelForCausalLM

import ensembles

In [4]:
current_device = torch.cuda.current_device()
print(f"Используется GPU с номером: {current_device}")

Используется GPU с номером: 0


In [5]:
# Проверяем доступность CUDA и количество GPU
# if torch.cuda.is_available():
#     if torch.cuda.device_count() > 1:  # Если есть более 1 GPU
#         device = torch.device("cuda:1")  # Явно указываем устройство 1
#         print(f"Переключились на GPU 1")
#     else:
#         print("Доступен только один GPU (номер 0), переключение невозможно")
# else:
#     print("CUDA недоступно")

In [6]:
device = torch.device("cuda")

In [7]:
print(f'Remaining GPU memory: {torch.cuda.memory_allocated() / (1024 ** 3):.2f} GB')

Remaining GPU memory: 0.00 GB


In [8]:
from sequential_models import GPTAdapterLayer

In [9]:
start_train = False#Запускаем ли мы сейчас обучение с нуля, или с какого-то чекпоинта
train_transformer = True
train_lm_head = True



cardinality = 128256#размер словаря токенов

padding_token = 128009
forbidden_tokens_list = [padding_token]
seed = int(np.random.rand() * 1000000)#random seed for data sampling

bits_per_number = 4

In [10]:
max_batch_size4loss = 1024*2
head_max_batch_size = 1024*2

In [11]:
#конфигурации моделек
mode = 'pretrain'#'finetune', 'pretrain'
model_version = "heavy_3"


if model_version == "light_1":
    embed_dim = 2048
    model_name = "unsloth/Llama-3.2-1B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 4
    t_layers_count = 2
    layer_configs = [2048] * 4
    batch_size = 2
    accum_batch = 2
    lr = 1e-3
    opt_type = 'adam'
    heavy_head = False
elif model_version == "heavy_1":
    embed_dim = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 2
    t_layers_count = 3
    layer_configs = [1024*3] * 6
    batch_size = 1
    accum_batch = 8000#1000
    lr = 1e-6
    conservativity = 1e-3
    opt_type = 'adam'
    heavy_head = False
elif model_version == "heavy_2":
    embed_dim = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 4
    t_layers_count = 3
    layer_configs = [1024*4] * 10
    batch_size = 1
    accum_batch = 10000#1000
    lr = 1e-6
    conservativity = 1e-3
    opt_type = 'adam'

    embedding_size = embed_dim
    heavy_head = False
    net_dropout_rate = 0
    individ_dropout_rate = 0.02
    layer_configs_head = [1024] * 2 #[2048] * 3 #+ [1024] * 4
    sample_features = 1
    composition_size = 1
    memnet_params = {'num_heads':12, 'query_size':128, 'num_key_values':320, 'value_size':256}
    use_memnets = True
elif model_version == "heavy_3":
    embed_dim = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 4
    t_layers_count = 2
    layer_configs = [1024*4] * 2
    batch_size = 1
    accum_batch = 3000#1000
    lr = 1e-7
    conservativity = 0.2
    dropout = 0.15
    opt_type = 'adam'

    embedding_size = embed_dim
    heavy_head = True
    net_dropout_rate = 0
    individ_dropout_rate = 0.02
    layer_configs_head = [2048] * 8 + [1024] * 9
    sample_features = 1
    composition_size = 1
    memnet_params = {'num_heads':12, 'query_size':128, 'num_key_values':320, 'value_size':256}
    use_memnets = False



    
elif model_version == "heavy_4":
    embed_dim = 4096
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
    num_heads_tlayer = 16
    ff_dim = embed_dim  * 4
    t_layers_count = 1
    layer_configs = [1024*4] * 2
    batch_size = 1
    accum_batch = 1000#1000
    lr = 1e-5
    conservativity = 3e-2
    dropout = 0.15
    opt_type = 'adam'

    embedding_size = embed_dim
    heavy_head = True
    net_dropout_rate = 0
    individ_dropout_rate = 0.02
    layer_configs_head = [2048] * 2 + [1024] * 2
    sample_features = 1
    composition_size = 1
    memnet_params = {'num_heads':12, 'query_size':128, 'num_key_values':320, 'value_size':256}
    use_memnets = False

In [12]:
path2tadapter = f"tadapter_{model_version}.pth"
path2lmhead = f"lmhead_{model_version}.pth"

In [13]:
tokenizer = AutoTokenizer.from_pretrained("unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit")#, cache_dir="D:\cache\huggingface\\"+ model_name)

if bits_per_number == 4:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4"#, bnb_4bit_compute_dtype=torch.bfloat16
    )
elif bits_per_number == 8:
    bnb_config = BitsAndBytesConfig(
        load_in_8bit=True, bnb_8bit_use_double_quant=True, bnb_8bit_quant_type="nf8"#, bnb_8bit_compute_dtype=torch.bfloat16
    )
else:
    bnb_config = None

tokenizer.pad_token = tokenizer.eos_token
model = AutoModelForCausalLM.from_pretrained(
              model_name,
              #device_map="auto",
              #device_map=device,
              #torch_dtype=torch.bfloat16,
              quantization_config=bnb_config,
              cache_dir="D:\cache\huggingface\\"+ model_name)


<>:21: SyntaxWarning: invalid escape sequence '\c'
<>:21: SyntaxWarning: invalid escape sequence '\c'
/tmp/ipykernel_1539717/2897674363.py:21: SyntaxWarning: invalid escape sequence '\c'
  cache_dir="D:\cache\huggingface\\"+ model_name)
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/home/ssdovgan/main/lib/python3.12/site-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


In [ ]:
def count_trainable_parameters(model: nn.Module) -> tuple[int, int]:
    trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
    frozen = sum(p.numel() for p in model.parameters() if not p.requires_grad)
    return trainable, frozen

In [14]:
def integrate_adapter(model, path2tadapter,path2lmhead, train_transformer, train_lm_head, heavy_head):

    # Замена последнего слоя.
    initial_layer = model.model.layers[-1]

    #создать шаблон модели
    model.model.layers[-1] = GPTAdapterLayer(initial_layer, 
                                             dim=128, 
                                             num_heads=2, 
                                             ff_dim=128, 
                                             t_layers_count=1, 
                                             layer_configs=[64],
                                             conservativity=1.,
                                             dropout=0.1).to(device)
    
    
    model.model.layers[-1].corrector_stack = torch.load(path2tadapter, weights_only=False).to(device)
    #model.lm_head = torch.load(path2lmhead, weights_only=False).to(device)
    if heavy_head:
        model.lm_head = torch.load(path2lmhead, weights_only=False).to(device)
    else:
        model.lm_head.load_state_dict(torch.load(path2lmhead))

    # Заморозка всей модели
    for param in model.parameters():
        param.requires_grad = False

    # Разморозка последнего блока
    if train_transformer:
        for param in model.model.layers[-1].corrector_stack.parameters():
            param.requires_grad = True
    #И lm-head
    if train_lm_head or train_transformer:
        model.lm_head.float()
    
    if train_lm_head:
        for param in model.lm_head.parameters():
            param.requires_grad = True
        
    if heavy_head:
        for param in model.lm_head.submodels[-1].parameters():
            param.requires_grad = False
    
    if train_transformer:
        model.model.layers[-1].transformer_float_mode = 32
    else:
        model.model.layers[-1].transformer_float_mode = 16
    if train_lm_head:
        model.model.layers[-1].lmhead_float_mode = 32
    elif train_transformer:
        model.model.layers[-1].lmhead_float_mode = 16
    
    trainable, frozen = count_trainable_parameters(model)
    print(f"Обучаемые: {trainable:,}\nЗамороженные: {frozen:,}")
    #return model

In [15]:
if start_train:
    # Исходная моделька


    # Замена последнего слоя. Тут трансформер, то можно и твой резнет... 
    initial_layer = model.model.layers[-1]
    
    #создать модель
    torch.manual_seed(1)
    np.random.seed(1)
    random.seed(1)
    
    model.model.layers[-1] = GPTAdapterLayer(initial_layer, 
                                             dim=embed_dim, 
                                             num_heads=num_heads_tlayer, 
                                             ff_dim=ff_dim, 
                                             t_layers_count=t_layers_count, 
                                             layer_configs=layer_configs,
                                             conservativity=conservativity,
                                             dropout=dropout).to(device)
    
    if heavy_head:
        torch.save(model.lm_head.weight, "lin_model.pth")
        head = ensembles.EResNetPro(input_size=embedding_size, 
               out_size=cardinality, 
               net_dropout_rate=net_dropout_rate, 
               individ_dropout_rate=individ_dropout_rate,
               layer_configs=layer_configs_head, 
               use_sigmoid_end=False, 
               use_batchnorm=True, 
               use_activation=True, 
               activation=nn.LeakyReLU(), 
               sample_features=sample_features, 
               composition_size=composition_size, 
               lin_bottleneck_size=None,
               lin_model_add=nn.Linear(embedding_size, cardinality).to(device),
               memnet_params=memnet_params,
               use_memnets=use_memnets,
               max_batch_size=4096,
               aggregation_by_mean=False).to(device)
        head.max_batch_size=head_max_batch_size
        head.submodels[-1].weight = torch.nn.Parameter(torch.load( "lin_model.pth").to(device).to(torch.float32))
        head.submodels[-1].weight.requires_grad = True
        model.lm_head = head

        # Заморозка всей модели
        for param in model.parameters():
            param.requires_grad = False
        
        # Разморозка последнего блока
        if train_transformer:
            for param in model.model.layers[-1].corrector_stack.parameters():
                param.requires_grad = True
        #И lm-head
        model.lm_head.float()
        
        if train_lm_head:
            for param in model.lm_head.parameters():
                param.requires_grad = True
            
        if heavy_head:
            for param in model.lm_head.submodels[-1].parameters():
                param.requires_grad = False
        
        if train_transformer:
            model.model.layers[-1].transformer_float_mode = 32
        if train_lm_head:
            model.model.layers[-1].lmhead_float_mode = 32
        
        
        trainable, frozen = count_trainable_parameters(model)
        print(f"Обучаемые: {trainable:,}\nЗамороженные: {frozen:,}")
else:
    integrate_adapter(model, path2tadapter,path2lmhead, train_transformer, train_lm_head, heavy_head)

use_memnets True
Обучаемые: 2,401,041,221
Замороженные: 4,540,728,576
Обучаемые: 2,401,041,221
Замороженные: 4,540,728,576


In [16]:
if opt_type == 'adam':
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
else:
    momentum = 0.
    optimizer = torch.optim.SGD(model.parameters(), lr=lr*0.01, momentum=momentum)

In [17]:
class InstructDatasetR(Dataset):
    def __init__(self, data_file, tokenizer, max_seq_length, cut=None):
        self.data_file = data_file
        self.tokenizer = tokenizer
        self.max_seq_length = max_seq_length
        self.cut = cut
        self.data = self.load_data()
        self.log_samples = deque(maxlen=45)

    def load_data(self):
        with open(self.data_file, 'rb') as f:
            data = pickle.load(f)
        random.seed(seed)
        data = random.sample(data, len(data))
        # if self.cut is None:
        #     data = [data_cur for data_cur in data if len(data_cur[1])>5]
        # else:
        #     data = [data_cur for data_cur in data[:self.cut] if self.cut>5]
        # for i in range(100):
        #     print('***', data[i])
        #     try:
        #         if (not 'Алис' in data[i][0]) and (not 'Элеон"' in data[i][0]) and (not 'Софи"' in data[i][0]) and (not 'Сэм"' in data[i][0]) (not 'Алис' in data[i][1]) and (not 'Элеон"' in data[i][1]) and (not 'Софи"' in data[i][1]) and (not 'Сэм"' in data[i][1]):
        #             print('***', data[i])
        #     except Exception:
        #         pass
        return data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        self.log_samples.append(self.data[idx])
        parts = self.data[idx]
        text = parts[0]
        label = parts[1]
        parts = [parts[0], parts[1], 1]
        for r_variant in [-2,-1,-0.5,0.5,1,2]:
            s = f"<r{r_variant}>"
            if s in label:
                parts[-1] = r_variant
                label = label.replace(s, '')
                break
        r = parts[-1]
        if text is None:
            text = label
        
        # Кодируем текст и метку с помощью tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoding['input_ids']
        attention_mask = encoding['attention_mask']
        
        # Кодируем метку
        label_encoding = self.tokenizer.encode_plus(
            label,
            max_length=self.max_seq_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        label_ids = label_encoding['input_ids']
        return {
            'input_ids': input_ids[0],
            'attention_mask': attention_mask,
            'labels': label_ids[0],
            'mult': torch.tensor(r)
        }

In [18]:
#настройки датасета

if mode == 'pretrain':
    #претрейн
    max_seq_len = 6900
    max_seq_len4inference = 6850
    # max_seq_len = 4500
    # max_seq_len4inference = 4450
    # max_seq_len = 3800
    # max_seq_len4inference = 3750
    # max_seq_len = 3600
    # max_seq_len4inference = 3550
else:
    #дообучение
    # max_seq_len = 1950
    # max_seq_len4inference = 1900
    max_seq_len = 2050
    max_seq_len4inference = 2000

print('batch_size', batch_size)

batch_size 1


In [19]:
# Создание Dataset
while 1:
    try:
        if mode == 'pretrain':
            #dataset = InstructDatasetR("./data/dataset_llm_full_mflat.pkl", tokenizer, max_seq_len, cut=None)
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_flat.pkl", tokenizer, max_seq_len, cut=None)
        else:
            dataset = InstructDatasetR("../data_tea/dataset_llm_full_instruct.pkl", tokenizer, max_seq_len, cut=None)
        break
    except Exception as e:
        print(e, pd.Timestamp.now())
        time.sleep(10)
len(dataset)

1071219

In [20]:
def soft_clip(tens, mn, mx):
    mn = torch.tensor(mn).to(tens.device)
    mx = torch.tensor(mx).to(tens.device)
    tens[tens<mn] = torch.nn.Sigmoid()(tens[tens<mn]) + mn
    tens[tens>mx] = torch.nn.Sigmoid()(tens[tens>mx]) + mx
    return tens
# Функция для подсчёта количества последних токенов, равных padding_token
def count_padding(tensor, padding_token):
    counts = []
    for row in tensor:
        count = 0
        for token in reversed(row):
            if token.item() == padding_token:
                count += 1
            else:
                break
        counts.append(count)
    return min(counts)

In [21]:
loss_fct = nn.CrossEntropyLoss(reduction = 'none')

In [22]:
proba_drop_idx = 0.04

In [ ]:
print('batch_size', batch_size)
i_pointer = 0

batch_accum_counter = 0
loss_array = []
metr_array = []
while 1: 
    input_ids_full = []
    labels_full = []
    weights_full = []
    #for i in range(batch_size):
    i = 0
    offset_counter = 0
    while i < batch_size:
        offset_counter += 1
        sample_cur  = dataset[i_pointer + i]
        input_ids = sample_cur['input_ids']
        labels = sample_cur['labels']
        weights = sample_cur['mult']
        
        input_ids_len = len(input_ids)
        labels_len = len(labels)
        input_ids = input_ids[input_ids!=padding_token]
        
        drop_idx = torch.rand_like(input_ids.to(torch.float)) < proba_drop_idx
        input_ids = input_ids[~drop_idx]
        
        labels = labels[labels!=padding_token]
        padding_size = input_ids_len + labels_len - len(input_ids) - len(labels)
        input_ids_cur = torch.cat([input_ids, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        labels_cur = torch.cat([input_ids * 0 + padding_token, labels, torch.zeros(padding_size, dtype=torch.int32) + padding_token])[:max_seq_len4inference]
        #print('fact_size', np.min([len(input_ids) + len(labels), max_seq_len4inference]), len(input_ids) + len(labels))
        if torch.all(labels_cur) == padding_token:
            print('empty label')
            continue
        i += 1
        
        input_ids_full.append(input_ids_cur[:-1])
        labels_full.append(labels_cur[1:])
        weights_full.append(weights)

        # if not torch.any(input_ids!=0):
        #     #X + Y
        #     print(list(input_ids_cur.numpy()))
        #     print(list(labels_cur.numpy()))
        #     1/0
    i_pointer += i #should be batch_size
    #print('A', pd.Timestamp.now())
    
    input_ids_full = torch.stack(input_ids_full).to('cuda')
    labels_full = torch.stack(labels_full).to('cuda')
    weights_full = torch.stack(weights_full).to('cuda').T

    cnt_pads = count_padding(labels_full, padding_token)
    if cnt_pads >= input_ids_full.shape[1] - 5:
        continue
    if cnt_pads > 0:
        input_ids_full = input_ids_full[:, :-cnt_pads]
        labels_full = labels_full[:, :-cnt_pads]
            
        outp = model.forward(input_ids_full)
        logits = outp['logits']
        del outp
        #outp['states'] = outp['hidden_states'][-1]
    # {"loss": loss_agg, "logits": logits, "states":embeddings}
        #outp['states'] = outp['states'][:, :input_ids_full.shape[1]]
        #state_cur = outp['states'].reshape([outp['states'].shape[0] * outp['states'].shape[1], outp['states'].shape[2]])#.cpu().to(torch.float16).numpy()
        weights_full2d = torch.vstack([weights_full] * labels_full.shape[-1]).to('cuda').T
        labels_cur = labels_full.ravel()#.cpu().numpy()
        weights_cur = weights_full2d.ravel()

        del labels_full
        del weights_full2d
        del weights_full
        idx = torch.isin(labels_cur, torch.tensor(forbidden_tokens_list).to(device))
        labels_cur = labels_cur[~idx]
        #state_cur = state_cur[~idx]
        weights_cur = weights_cur[~idx]
    else:
        continue
    if labels_cur.shape[0] == 0:
        continue

    for batch_start in range(0, logits.shape[0], max_batch_size4loss):
        ce_loss = loss_fct(logits.view([logits.shape[0] * logits.shape[1], logits.shape[2]])[~idx][batch_start : batch_start + max_batch_size4loss], labels_cur.view(-1)[batch_start : batch_start + max_batch_size4loss])
        ce_loss = ce_loss * weights_cur[batch_start : batch_start + max_batch_size4loss].to('cuda')
        ce_loss = soft_clip(ce_loss, -0.6, 10.)
        loss = torch.mean(ce_loss)
        loss.backward()
        torch.cuda.empty_cache()
    # ce_loss = loss_fct(logits.view([logits.shape[0] * logits.shape[1], logits.shape[2]])[~idx], labels_cur.view(-1))
    # ce_loss = ce_loss * weights_cur.to('cuda')
    # ce_loss = soft_clip(ce_loss, -0.6, 10.)
    # loss = torch.mean(ce_loss)
    # loss.backward()
    
    #print('D', pd.Timestamp.now())

    acc = torch.mean((torch.argmax(logits.view([logits.shape[0] * logits.shape[1], logits.shape[2]])[~idx], axis=-1) == labels_cur).to(torch.float16))
    del logits
    del labels_cur
    batch_accum_counter += 1
    loss_array += [torch.mean(ce_loss).item()]
    metr_array += [acc.item()]
    if batch_accum_counter >= accum_batch:
        optimizer.step()
        optimizer.zero_grad()
        torch.cuda.empty_cache()
        print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())
        loss_array = []
        metr_array = []
        batch_accum_counter = 0

        
        try:
            if (np.random.rand() < 1 and accum_batch > 100) or (np.random.rand() < 0.1 and accum_batch <= 100):
                model.lm_head.training = False
                torch.save(model.model.layers[-1].corrector_stack, path2tadapter)
                if heavy_head:
                    torch.save(model.lm_head, path2lmhead)
                else:
                    torch.save(model.lm_head.state_dict(), path2lmhead)
    
                if np.random.rand()<0.2:
                    torch.save(model.model.layers[-1].corrector_stack, path2tadapter + '.back')
                    if heavy_head:
                        torch.save(model.lm_head, path2lmhead + '.back')
                    else:
                        torch.save(model.lm_head.state_dict(), path2lmhead + '.back')
                model.lm_head.training = True
        except Exception:
            pass

batch_size 1


/tmp/ipykernel_1539717/2664646116.py:52: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matrices or `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3683.)
  weights_full = torch.stack(weights_full).to('cuda').T


ce_loss 1.778 5024 from 1071219 0.469 % acc 0.6626245524088542 2025-04-20 12:43:16.201619
ce_loss 1.76 10125 from 1071219 0.94518 % acc 0.6639802856445313 2025-04-20 13:06:47.375269
ce_loss 1.756 15178 from 1071219 1.41689 % acc 0.6695648803710937 2025-04-20 13:30:03.021425
ce_loss 1.747 20411 from 1071219 1.9054 % acc 0.6697180989583333 2025-04-20 13:54:21.487592
ce_loss 1.727 25448 from 1071219 2.37561 % acc 0.6693824462890625 2025-04-20 14:18:39.164935
ce_loss 1.701 30563 from 1071219 2.8531 % acc 0.6723819580078125 2025-04-20 14:42:20.754488


In [ ]:
#батч = 4, аккум = 20
#53 сек
#батч = 2, аккум = 40
#42 сек
#батч = 1, аккум = 80
#35 сек

In [ ]:
#если принудительно стопнули, можно добить батч
optimizer.step()
optimizer.zero_grad()
torch.cuda.empty_cache()
print('ce_loss', np.round(np.mean(loss_array), 3), i_pointer, 'from', len(dataset), np.round(100 * i_pointer/len(dataset), 5), '%', 'acc', np.mean(metr_array), pd.Timestamp.now())

In [ ]:
#Засейвить
model.lm_head.training = False
torch.save(model.model.layers[-1].corrector_stack, path2tadapter)
if heavy_head:
    torch.save(model.lm_head, path2lmhead)
else:
    torch.save(model.lm_head.state_dict(), path2lmhead)
model.lm_head.training = True

In [ ]:
#check generation

model.model.layers[-1].half()
model.lm_head.half()
model.model.layers[-1].transformer_float_mode = 16
model.model.layers[-1].lmhead_float_mode = 16

prompt = 'Твоя роль: Сталин. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
#prompt = 'Твоя роль: Девушка-флористка. \n Серёга: Какая фракция в игре Герои Меча и Магии 3 соответствует твоей личности? Your answer, ONLY ENGLISH (end by "|"):'
#prompt = 'Твоя роль: Data scientist. \n Серёга: Мне надо улучшить LLM. У меня ресурсы на запуск Llama 3.1 8B или Qwen 2.5 14B, квантованные до 4 бит. Мне нужно, чтобы мои модели былу умнее (на уровне 70 B), но при этом имели настолько же низкие требования по памяти и запускались так же быстро. Я готов обучать новую модель. Я готов пробовать другие архитектуры. Опиши план, что делать. Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Инженер-ракетчик. \n Серёга: Мне надо сделать ракету, в домашних условиях, небольшую. Расскажи, как это провернуть? ТВОЙ ОТВЕТ НА РУССКОМ  (end by "|"):'
#prompt = 'Твоя роль: Геймер, студент-инженер. \n Серёга: Перечисли всех монстров из Doom, каких вспомнишь! Your answer, ONLY ENGLISH  (end by "|"):'
#prompt = 'Твоя роль: Гермиона Грейнджер. \n Серёга: Как справиться с василиском? Your answer, ONLY ENGLISH  (end by "|", 200 tokens):'
inputs = tokenizer(prompt, return_tensors="pt").to(device)
repetition_penalty = 1.2
t = pd.Timestamp.now()
generate_ids = model.generate(inputs.input_ids.to(device),
                                        #attention_mask=inputs["attention_mask"],
                                        max_new_tokens=200,
                                        temperature=0.01,
                                        pad_token_id=tokenizer.pad_token_id,
                                        eos_token_id=tokenizer.eos_token_id,
                                        bos_token_id=tokenizer.bos_token_id,
                                        #no_repeat_ngram_size=3,
                                        repetition_penalty=repetition_penalty,
                                        early_stopping=True,
                                        use_cache=True,
                                        num_beams=1,
                                        tokenizer=tokenizer)
print(pd.Timestamp.now() - t)
answer = tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]
print(answer)